In [17]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

tar: spark-3.5.0-bin-hadoop3.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, max, min, count, to_date, stddev
from pyspark.sql.types import DoubleType, IntegerType
import requests
import json

spark = (
    SparkSession.builder
    .appName("BC_API_Expectativas_Selic")
    .getOrCreate()
)

In [19]:
url = "https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/ExpectativasMercadoSelic?$top=1000&$format=json&$select=Indicador,Data,Reuniao,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo"


### Criando o DataFrame

In [20]:
response = requests.get(url)
dados_json = response.json()["value"]

In [21]:
# Criando o DataFrame Spark diretamente de um objeto Python
df = spark.createDataFrame(dados_json)
df.show(5)
df.printSchema()

+----------+------------+---------+------+-------+-------+------+-------+-----------+------------------+
|      Data|DesvioPadrao|Indicador|Maximo|  Media|Mediana|Minimo|Reuniao|baseCalculo|numeroRespondentes|
+----------+------------+---------+------+-------+-------+------+-------+-----------+------------------+
|2025-10-31|      1.0116|    Selic|  13.0|10.7112|   10.5|   8.5|R6/2027|          1|                58|
|2025-10-31|      0.9478|    Selic|  13.0|10.7083|   10.5|   8.5|R6/2027|          0|               102|
|2025-10-31|      0.9606|    Selic|  13.0|10.8712|  10.75|  8.75|R5/2027|          1|                66|
|2025-10-31|      0.9344|    Selic|  13.0|10.8684|   11.0|  8.75|R5/2027|          0|               114|
|2025-10-31|      0.9022|    Selic|  13.0|11.0373|   11.0|   9.0|R4/2027|          1|                67|
+----------+------------+---------+------+-------+-------+------+-------+-----------+------------------+
only showing top 5 rows

root
 |-- Data: string (nullab

## Transformações

Converter coluna "Data" para tipo date

In [22]:
df = df.withColumn("Data", to_date(col("Data"), "yyyy-MM-dd"))
df.show(10)

+----------+------------+---------+------+-------+-------+------+-------+-----------+------------------+
|      Data|DesvioPadrao|Indicador|Maximo|  Media|Mediana|Minimo|Reuniao|baseCalculo|numeroRespondentes|
+----------+------------+---------+------+-------+-------+------+-------+-----------+------------------+
|2025-10-31|      1.0116|    Selic|  13.0|10.7112|   10.5|   8.5|R6/2027|          1|                58|
|2025-10-31|      0.9478|    Selic|  13.0|10.7083|   10.5|   8.5|R6/2027|          0|               102|
|2025-10-31|      0.9606|    Selic|  13.0|10.8712|  10.75|  8.75|R5/2027|          1|                66|
|2025-10-31|      0.9344|    Selic|  13.0|10.8684|   11.0|  8.75|R5/2027|          0|               114|
|2025-10-31|      0.9022|    Selic|  13.0|11.0373|   11.0|   9.0|R4/2027|          1|                67|
|2025-10-31|      0.8868|    Selic|  13.0|11.0556|   11.0|   9.0|R4/2027|          0|               117|
|2025-10-31|      0.8189|    Selic|  13.0|11.2612|  11.

Correção dos tipos numéricos enviados como string

In [23]:
df = (
    df.withColumn("Media", col("Media").cast(DoubleType()))
      .withColumn("Mediana", col("Mediana").cast(DoubleType()))
      .withColumn("DesvioPadrao", col("DesvioPadrao").cast(DoubleType()))
      .withColumn("Minimo", col("Minimo").cast(DoubleType()))
      .withColumn("Maximo", col("Maximo").cast(DoubleType()))
      .withColumn("numeroRespondentes", col("numeroRespondentes").cast(IntegerType()))
)
df.show(10)

+----------+------------+---------+------+-------+-------+------+-------+-----------+------------------+
|      Data|DesvioPadrao|Indicador|Maximo|  Media|Mediana|Minimo|Reuniao|baseCalculo|numeroRespondentes|
+----------+------------+---------+------+-------+-------+------+-------+-----------+------------------+
|2025-10-31|      1.0116|    Selic|  13.0|10.7112|   10.5|   8.5|R6/2027|          1|                58|
|2025-10-31|      0.9478|    Selic|  13.0|10.7083|   10.5|   8.5|R6/2027|          0|               102|
|2025-10-31|      0.9606|    Selic|  13.0|10.8712|  10.75|  8.75|R5/2027|          1|                66|
|2025-10-31|      0.9344|    Selic|  13.0|10.8684|   11.0|  8.75|R5/2027|          0|               114|
|2025-10-31|      0.9022|    Selic|  13.0|11.0373|   11.0|   9.0|R4/2027|          1|                67|
|2025-10-31|      0.8868|    Selic|  13.0|11.0556|   11.0|   9.0|R4/2027|          0|               117|
|2025-10-31|      0.8189|    Selic|  13.0|11.2612|  11.

Filtrar somente linhas com mais de 5 respondentes

In [24]:
df = df.filter(col("numeroRespondentes") > 5)
df.show(10)

+----------+------------+---------+------+-------+-------+------+-------+-----------+------------------+
|      Data|DesvioPadrao|Indicador|Maximo|  Media|Mediana|Minimo|Reuniao|baseCalculo|numeroRespondentes|
+----------+------------+---------+------+-------+-------+------+-------+-----------+------------------+
|2025-10-31|      1.0116|    Selic|  13.0|10.7112|   10.5|   8.5|R6/2027|          1|                58|
|2025-10-31|      0.9478|    Selic|  13.0|10.7083|   10.5|   8.5|R6/2027|          0|               102|
|2025-10-31|      0.9606|    Selic|  13.0|10.8712|  10.75|  8.75|R5/2027|          1|                66|
|2025-10-31|      0.9344|    Selic|  13.0|10.8684|   11.0|  8.75|R5/2027|          0|               114|
|2025-10-31|      0.9022|    Selic|  13.0|11.0373|   11.0|   9.0|R4/2027|          1|                67|
|2025-10-31|      0.8868|    Selic|  13.0|11.0556|   11.0|   9.0|R4/2027|          0|               117|
|2025-10-31|      0.8189|    Selic|  13.0|11.2612|  11.

Criar coluna nova "Amplitude" (Maximo - Minimo)

In [25]:
df = df.withColumn("Amplitude", col("Maximo") - col("Minimo"))
df.show(10)

+----------+------------+---------+------+-------+-------+------+-------+-----------+------------------+---------+
|      Data|DesvioPadrao|Indicador|Maximo|  Media|Mediana|Minimo|Reuniao|baseCalculo|numeroRespondentes|Amplitude|
+----------+------------+---------+------+-------+-------+------+-------+-----------+------------------+---------+
|2025-10-31|      1.0116|    Selic|  13.0|10.7112|   10.5|   8.5|R6/2027|          1|                58|      4.5|
|2025-10-31|      0.9478|    Selic|  13.0|10.7083|   10.5|   8.5|R6/2027|          0|               102|      4.5|
|2025-10-31|      0.9606|    Selic|  13.0|10.8712|  10.75|  8.75|R5/2027|          1|                66|     4.25|
|2025-10-31|      0.9344|    Selic|  13.0|10.8684|   11.0|  8.75|R5/2027|          0|               114|     4.25|
|2025-10-31|      0.9022|    Selic|  13.0|11.0373|   11.0|   9.0|R4/2027|          1|                67|      4.0|
|2025-10-31|      0.8868|    Selic|  13.0|11.0556|   11.0|   9.0|R4/2027|       

Agrupamento estatístico por Reunião

In [26]:
df = (
    df.groupBy("Reuniao")
       .agg(
           avg("Media").alias("Media_Da_Media"),
           avg("Mediana").alias("Media_Da_Mediana"),
           stddev("Media").alias("Desvio_Medio"),
           max("Maximo").alias("Maior_Maximo"),
           min("Minimo").alias("Menor_Minimo")
       )
)
df.show(10)

+-------+------------------+------------------+--------------------+------------+------------+
|Reuniao|    Media_Da_Media|  Media_Da_Mediana|        Desvio_Medio|Maior_Maximo|Menor_Minimo|
+-------+------------------+------------------+--------------------+------------+------------+
|R5/2027|10.856507812499999|      10.794921875| 0.05658827716585131|        13.5|        8.75|
|R6/2026|12.687590322580649|12.705645161290322| 0.03783196544894755|        15.0|        10.0|
|R3/2027|       11.32153125|       11.37890625| 0.04663125484663079|        13.5|         9.5|
|R7/2026|12.417145161290327|12.487903225806452| 0.03572976294085381|        15.0|        10.0|
|R7/2025|14.999504838709676|              15.0|0.001585738429565...|        15.1|        14.5|
|R1/2027|11.903799999999997|              12.0| 0.03515082257741409|        13.5|         9.5|
|R5/2026|13.048217741935485|              13.0| 0.03840719930887999|        15.0|        10.0|
|R8/2025|14.937711290322582|              15.0|0.0

## Ações

Inspecionar se as transformações foram feitas corretamente.

In [27]:
df.show(20, truncate=False)

+-------+------------------+------------------+---------------------+------------+------------+
|Reuniao|Media_Da_Media    |Media_Da_Mediana  |Desvio_Medio         |Maior_Maximo|Menor_Minimo|
+-------+------------------+------------------+---------------------+------------+------------+
|R5/2027|10.856507812499999|10.794921875      |0.05658827716585131  |13.5        |8.75        |
|R6/2026|12.687590322580649|12.705645161290322|0.03783196544894755  |15.0        |10.0        |
|R3/2027|11.32153125       |11.37890625       |0.04663125484663079  |13.5        |9.5         |
|R7/2026|12.417145161290327|12.487903225806452|0.03572976294085381  |15.0        |10.0        |
|R7/2025|14.999504838709676|15.0              |0.0015857384295659936|15.1        |14.5        |
|R1/2027|11.903799999999997|12.0              |0.03515082257741409  |13.5        |9.5         |
|R5/2026|13.048217741935485|13.0              |0.03840719930887999  |15.0        |10.0        |
|R8/2025|14.937711290322582|15.0        

Contagem total de registros

In [28]:
df2 = spark.createDataFrame(dados_json)
df2.show(5)
df2.printSchema()

+----------+------------+---------+------+-------+-------+------+-------+-----------+------------------+
|      Data|DesvioPadrao|Indicador|Maximo|  Media|Mediana|Minimo|Reuniao|baseCalculo|numeroRespondentes|
+----------+------------+---------+------+-------+-------+------+-------+-----------+------------------+
|2025-10-31|      1.0116|    Selic|  13.0|10.7112|   10.5|   8.5|R6/2027|          1|                58|
|2025-10-31|      0.9478|    Selic|  13.0|10.7083|   10.5|   8.5|R6/2027|          0|               102|
|2025-10-31|      0.9606|    Selic|  13.0|10.8712|  10.75|  8.75|R5/2027|          1|                66|
|2025-10-31|      0.9344|    Selic|  13.0|10.8684|   11.0|  8.75|R5/2027|          0|               114|
|2025-10-31|      0.9022|    Selic|  13.0|11.0373|   11.0|   9.0|R4/2027|          1|                67|
+----------+------------+---------+------+-------+-------+------+-------+-----------+------------------+
only showing top 5 rows

root
 |-- Data: string (nullab

In [29]:
df2.count()

1000

Estatísticas descritivas

In [31]:
df2.describe(["Media", "Mediana"]).show()

+-------+------------------+------------------+
|summary|             Media|           Mediana|
+-------+------------------+------------------+
|  count|              1000|              1000|
|   mean|12.743872799999982|          12.75525|
| stddev| 1.453579541705312|1.4946556444530945|
|    min|             10.25|              10.5|
|    max|           15.0007|              15.0|
+-------+------------------+------------------+



Traz os dados para o driver

In [32]:
resultado_local = df2.collect()

Salvando os dados em arquivo (CSV)

In [33]:
df2.write.mode("overwrite").csv("/content/saida_selic")